## Include
The environment with all the packages can be activated by typing `julia` and then `]`and then typing `activate BeyondHulten`

In [1]:
using MAT, LinearAlgebra, Random , NonlinearSolve
using StaticArrays, Distributed, SciMLNLSolve, LineSearches
using JLD
using Statistics
using SharedArrays
include("src/mvnrnd.jl")

mvnrnd (generic function with 1 method)

## Data Loading
Loads in the big data matrix and the stfp matrix created by matlab

In [2]:
function loadInData()
    file = matopen("data/simulationData.mat")

    data = read(file, "data")

    close(file)

    return data
end

function loadStfp()
    file = matopen("data/stfp.mat")

    stfp = read(file,"stfp")

    close(file)

    return stfp
end

loadStfp (generic function with 1 method)

## Data processing
Processes the data just as in the matlab version

In [3]:
data = loadInData()

grossy = reshape(data[:, 3], 46, 88)'
capital = reshape(data[:, 4], 46, 88)'; #NOMINAL CAPITAL
labor = reshape(data[:, 5], 46, 88)'; #NOMINAL LABOR
vadd = labor + capital; #NOMINAL VALUE;

removableSectors = [60, 80:88...];
grossy = grossy[setdiff(1:end, removableSectors), :];
capital = capital[setdiff(1:end, removableSectors), :];
labor = labor[setdiff(1:end, removableSectors), :];
vadd = vadd[setdiff(1:end, removableSectors), :];

grossSales = vec((sum(grossy, dims=2) .!= 0));
grossy = grossy[grossSales, :];
capital = capital[grossSales, :];
labor = labor[grossSales, :];
vadd = vadd[grossSales, :];

stfp = loadStfp();
Σ = cov(stfp, dims=2);
μ = mean(stfp, dims=2);

## Creating usable variables
Function that creates the metrics for a given year

In [4]:
function getVariables(year)
    IO = data[data[:, 1].==year, :]
    temp = [8, 60, 62, 80:88...]
    IO = IO[setdiff(1:end, temp), setdiff(1:end, [1, 2, 3, 4, 5, 94])]
    IO = IO[:, setdiff(1:end, temp)]
    Ω = IO ./ sum(IO, dims=2)
    α = vadd[:, year-1959] ./ grossy[:, year-1959]
    β = ((I - diagm(1 .- α) * Ω)' * grossy[:, year-1959])
    @views β[β.<0] .= 0
    β = β / sum(β)
    λ = (inv(I - diagm(1 .- α) * Ω)' * β)
    L = λ .* α

    return α, β, Ω, L, λ

end

getVariables (generic function with 1 method)

## Elasticities

In [5]:
ε = .5;
θ = 0.001;
σ = .9;

## Objective Function

Here the objective function in `SimulationDerivs.m` is formulated, also the Jacobian is given, to helpt the solver.

In [6]:
function problem(X :: AbstractVector{T}, A, β, Ω, α, ε, θ, σ,L) where T
    N = length(α)
    p = X[1:N]
    y = X[N+1:end]
    
    Out :: Vector{T} = zeros(eltype(X),2*N)
    
    q = (Ω * p .^ (1-θ)) .^ (1 / (1 - θ))
    w = p .* (A .^ ((ε - 1)/ε)) .* (α .^ (1 / ε)) .* (y .^ (1/ε)) .* L .^ (-1/ε)
    C = w' * L
  
    Out[1:N] = p - (A .^ (ε - 1) .* (α .* w .^ (1- ε) + (1 .- α) .* q .^ (1 - ε))) .^ (1/(1-ε))
    Out[N+1:end] = y' - y' * diagm(p)^ε * diagm(A)^(ε-1) * diagm(q)^(θ-ε) * diagm(1 .- α) * Ω * diagm(p)^(-θ) - β'*diagm(p)^(-σ)*C
    
    return Out
end

problem (generic function with 1 method)

In [7]:
function problemJacobian(X, A, β, Ω, α, ε, θ, σ,L)
    N = length(α)
    p = @view X[1:N]
    y = @view X[N+1:end]
    
    Out = zeros(eltype(X),2*N)
    
    q = (Ω * p .^ (1-θ)) .^ (1 / (1 - θ))
    w = p .* (A .^ ((ε - 1)/ε)) .* (α .^ (1 / ε)) .* (y .^ (1/ε)) .* L .^ (-1/ε)
    C = w' * L
  
    
    DQDP = (q .^ θ * (p.^(-θ))') .* Ω;
    DWDP = diagm(A.^((ε-1)/ε) .* α.^(1/ε) .* y.^(1/ε) .* (1 ./ L).^(1/ε));
    DWDY = (1/ε) * diagm(p .* A.^((ε-1)/ε) .* α.^(1/ε).* (y.^(1/ε-1)) .* L.^(-1/ε));
    DCDP = DWDP' * L; 
    DCDY = DWDY' * L;

    DOut1DP = I - diagm(diagm(A)^(-1) * ((α .* (w .^ (1-ε)) + (1 .- α) .* (q.^(1-ε)))).^(ε/(1-ε))) * (diagm(α) * diagm(w.^(-ε))* DWDP + diagm(1 .-α)*diagm(q.^(-ε))*DQDP);

    DOut1DY =  -diagm(diagm(A.^(-1))*((α.*(w.^(1-ε))+(1 .- α).*(q.^(1-ε)))).^(ε/(1-ε)))*(diagm(α)*diagm(w.^(-ε))*DWDY);

    DOut2DP = -(ε * diagm(p.^(-θ))*Ω'*diagm((p.^(ε-1)).*(y).*(q.^(θ-ε)).*(1 .- α).*(A.^(ε-1)))
        + (θ-ε)*diagm(p.^(-θ))*Ω'*diagm((p.^(ε)).*(y).*(q.^(θ-ε-1)).*(1 .- α).*(A.^(ε-1)))*DQDP 
        - σ*diagm(β.*p.^(-σ-1))*C + broadcast(*, β.*(p.^(-σ)), DCDP')
        - θ* diagm(p.^(-θ-1)).*diagm(Ω'*diagm((p.^(ε)).*(q.^(θ-ε)).*(1 .- α).*(A.^(ε-1)))*y));

    DOut2DY = I - (diagm(p)^ε*diagm(A)^(ε-1)*diagm(q)^(θ-ε)*diagm(1 .- α)*Ω*diagm(p)^(-θ))' - broadcast(*, β.*(p.^(-σ)), DCDY');
    OutDeriv = [DOut1DP DOut1DY; DOut2DP DOut2DY];

end

problemJacobian (generic function with 1 method)

## Generating Random Shocks

In [8]:
function generateRandomShock(Σ,α,Ω,λ)
    A = exp.(mvnrnd(-1/2*diag(Σ),diagm(diag(Σ))));
    
    init = Complex.([exp.(-inv(I - diagm(1 .- α)*Ω)*log.(A))..., λ ./exp.(-inv(I - diagm(1 .- α) * Ω)*log.(A))...]); 
    (A,init)
end

generateRandomShock (generic function with 1 method)

## Solving for shocks

In [9]:

trials = 100;
GDP = SharedVector(zeros(trials))
λ_sim = SharedMatrix(zeros(76, trials))
α, β, Ω, L, λ = getVariables(1973);
f = NonlinearFunction((u, p) -> problem(u, p...), jac=(u, p) -> problemJacobian(u, p...))

@distributed for k in 1:trials

    A, init = generateRandomShock(Σ, α, Ω, λ)

    p = [A, β, Ω, α, ε, θ, σ, L]

    ProbN = NonlinearProblem(f, init, p)
    sol = solve(ProbN, NLSolveJL(linesearch=HagerZhang(), method=:newton), reltol=1e-8, abstol=1e-8)

    x = real.(sol.u)

    p = @view x[1:76]
    q = @view x[77:end]
    GDP[k] = (p .* (A .^ ((ε - 1) / ε)) .* (α .^ (1 / ε)) .* (q .^ (1 / ε)) .* L .^ (-1 / ε))' * L
    λ_sim[:, k] .= (p .* q) ./ GDP[k]

end

Task (done) @0x00007f4268797d00

## Solving for a progessive sector shock

In [10]:
function shockSectorProgressively(sector, shocks, year=1983)

    (α, β, Ω, L, λ) = getVariables(year)

    x0 = Complex.([ones(76)..., λ...])
    f = NonlinearFunction((u, p) -> problem(u, p...), jac=(u, p) -> problemJacobian(u, p...))
    GDP = []
    for shock in shocks
        A = ones(76)
        A[sector] = shock

        p = [A, β, Ω, α, ε, θ, σ, L]
        ProbN = NonlinearProblem(f, x0, p)

        #sol = solve(ProbN,NewtonRaphson(), reltol = 1e-10,abstol = 1e-8)
        sol = solve(ProbN, NLSolveJL(method=:newton, linesearch=BackTracking()), reltol=1e-10, abstol=1e-8)

        x = sol.u

        p = real.(x[1:76])
        y = real.(x[77:152])
        append!(GDP, (p .* (A .^ ((ε - 1) / ε)) .* (α .^ (1 / ε)) .* (y .^ (1 / ε)) .* L .^ (-1 / ε))' * L)

        x0 = x
    end

    return GDP

end

shockSectorProgressively (generic function with 2 methods)

In [11]:
years = 1980:2000
M = 50

a = 0.7;
b = 1.3;

sectors = 1:60

GDP = SharedArray(zeros(2 * M, length(sectors), length(years)))

@distributed for year in years
    for i in 1:length(sectors)
        GDP[1:M, i, year-(years[1]-1)] = shockSectorProgressively(sectors[i], LinRange(1, a, M), year)
        GDP[M+1:end, i, year-(years[1]-1)] = shockSectorProgressively(sectors[i], LinRange(1, b, M), year)
    end
end


In [ ]:

@views reverse!(GDP[1:M, :,:], dims=1)
GDP = GDP[setdiff(1:end, M), :,:]

In [ ]:
save("calulatedGSPS.jld","GDP",GDP);

In [ ]:
GDP = load("calulatedGSPS.jld","GDP");

## Visiualisation of progressive sector shocks.

In [ ]:
using GLMakie
Makie.inline!(false)
a = 0.7
b = 1.3
M = 50
fig = Figure()

ax = Axis(fig[2, 1])

sg = SliderGrid(fig[1, 1],
    (label = "Year", range = 1980:2000, startvalue = 5),
    (label = "Sector", range = 1:60, startvalue = 10)
)


indexYear = lift(sg.sliders[1].value) do x
    x - 1979 
end

indexSector = lift(sg.sliders[2].value) do x
    x
end

lineShowing = @lift(GDP[:,$indexSector,$indexYear])

lines!(LinRange(a,b,2*M-1),lineShowing)

fig